In [1]:
import os
import random
import numpy as np
import pandas as pd
import cudf, itertools
import scipy.sparse as ssp
from functools import lru_cache, partial
from tqdm import tqdm, trange
from collections import Counter, defaultdict
import datasets
from datasets import Dataset as TFDataset

# cut valid candidates 

In [5]:
merged_candidates_feature_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/XGBoost/candidates_phase2/merged_candidates_150_feature.parquet'
train_sessions_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/data_for_recstudio/task1_data/task13_4_task1_train_sessions_phase2.csv'
valid_sessions_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/data_for_recstudio/task1_data/task13_4_task1_valid_sessions_phase2.csv'
test_sessions_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/raw_data/sessions_test_task1_phase2.csv'
product_data_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/raw_data/products_train.csv'

In [3]:
TOPK = 250

In [6]:
merged_candidates = pd.read_parquet(merged_candidates_feature_path)

In [8]:
sasrec_text_predictions = merged_candidates.copy()
sasrec_text_predictions = sasrec_text_predictions.sort_values(['sess_id','sasrec_feat_scores_3'], ascending=[True, False]).reset_index(drop=True)
sasrec_text_predictions['n'] = sasrec_text_predictions.groupby('sess_id')['product'].cumcount().astype('int32')
merged_candidates_cut = sasrec_text_predictions.loc[sasrec_text_predictions['n'] < TOPK]

In [ ]:
merged_candidates_cut = merged_candidates_cut.sort_values(['sess_id', 'product']).reset_index(drop=True)

In [ ]:
merged_candidates_cut.to_parquet('/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/XGBoost/candidates_phase2/merged_candidates_cut_250_feature.parquet')

In [7]:
sasrec_text_predictions

,sess_id,sess_locale,product,target,sess_avg_price,product_price,product_freq,sasrec_scores_2,normalized_sasrec_scores_2,sasrec_scores_3,...,co_graph_counts_1,normalized_co_graph_counts_1,co_graph_counts_2,normalized_co_graph_counts_2,bert_scores,normalized_bert_scores,cos_bert_scores,sasrec_text_scores,normalized_sasrec_text_scores,n
0,0,UK,B06XGDZVZR,0.0,7.388571,9.990000,38.0,19.064575,2.745485e-01,17.880465,...,28.994047,0.134610,23.0,0.182540,483.273376,1.025510e-02,0.980086,21.670799,3.365566e-01,0
1,0,UK,B06XG1LZ6Z,1.0,7.388571,9.990000,30.0,19.223602,3.218723e-01,17.775932,...,25.950001,0.120477,19.0,0.150794,486.098450,1.729233e-01,0.984002,21.547813,2.976089e-01,1
2,0,UK,B06XGD9VLV,0.0,7.388571,10.990000,31.0,18.897261,2.322498e-01,17.897360,...,22.759523,0.105665,14.0,0.111111,486.158997,1.837167e-01,0.984541,20.899105,1.555663e-01,2
3,0,UK,B076PN1SKG,0.0,7.388571,12.790000,260.0,16.387152,1.887247e-02,15.293397,...,11.583333,0.053778,8.0,0.063492,481.965454,2.772785e-03,0.976309,18.912140,2.132984e-02,3
4,0,UK,B01MYUDYP7,0.0,7.388571,13.990000,93.0,16.480183,2.071246e-02,15.037660,...,10.117929,0.046974,7.0,0.055556,483.992310,2.104594e-02,0.980738,18.751011,1.815558e-02,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77570148,361580,DE,B096VBQ67C,0.0,32.424000,36.761604,129.0,0.000000,6.665982e-09,0.000000,...,0.333333,0.007712,0.0,0.000000,0.000000,0.000000e+00,0.000000,0.620562,9.517644e-10,179
77570149,361580,DE,B09LZ8VJT7,0.0,32.424000,36.761604,6.0,0.000000,6.665982e-09,0.000000,...,1.500000,0.034704,1.0,0.041667,0.000000,0.000000e+00,0.000000,0.620562,9.517644e-10,180
77570150,361580,DE,B07HP3VKVZ,0.0,32.424000,31.950001,8.0,4.817217,8.240525e-07,5.881405,...,0.000000,0.000000,0.0,0.000000,376.899109,1.060987e-09,0.923120,0.362109,7.349952e-10,181
77570151,361580,DE,B09YD12Q45,0.0,32.424000,16.900000,4.0,4.101108,4.026731e-07,5.103590,...,0.000000,0.000000,0.0,0.000000,378.540283,5.475995e-09,0.923060,-1.619148,1.013528e-10,182


In [8]:
merged_candidates_cut['target'].sum() / 361581

0.7065443151050526

In [9]:
merged_candidates_cut

,sess_id,sess_locale,product,target,sess_avg_price,product_price,product_freq,sasrec_scores_2,normalized_sasrec_scores_2,sasrec_scores_3,...,co_graph_counts_1,normalized_co_graph_counts_1,co_graph_counts_2,normalized_co_graph_counts_2,bert_scores,normalized_bert_scores,cos_bert_scores,sasrec_text_scores,normalized_sasrec_text_scores,n
0,0,UK,B000V599Y2,0.0,7.388571,5.200000,37.0,13.152878,7.433639e-04,10.677187,...,0.000000,0.000000,0.0,0.000000,476.148773,0.000008,0.965266,14.527957,2.660486e-04,99
1,0,UK,B007VZUA7U,0.0,7.388571,7.000000,36.0,9.393598,1.732076e-05,8.838863,...,0.000000,0.000000,0.0,0.000000,475.083313,0.000003,0.963742,13.181476,6.921363e-05,143
2,0,UK,B009EUAEQC,0.0,7.388571,7.490000,4.0,11.754339,1.835794e-04,10.670128,...,1.033333,0.004797,0.0,0.000000,476.230896,0.000009,0.964250,13.900270,1.420235e-04,124
3,0,UK,B00I0UKKD4,0.0,7.388571,17.049999,118.0,11.835367,1.990737e-04,11.681271,...,1.833333,0.008512,1.0,0.007937,482.222168,0.003584,0.978083,17.833698,7.254814e-03,10
4,0,UK,B015Q0MXM6,0.0,7.388571,13.970000,7.0,11.457336,1.364072e-04,10.151916,...,0.592857,0.002752,0.0,0.000000,482.106201,0.003192,0.977429,13.757364,1.231110e-04,127
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54224499,361580,DE,B09WR31N2M,0.0,32.424000,24.990000,2.0,5.252259,1.273185e-06,1.911197,...,0.000000,0.000000,0.0,0.000000,385.367645,0.000005,0.941466,9.536082,7.087460e-06,124
54224500,361580,DE,B0BB7XV97M,0.0,32.424000,47.990002,56.0,9.117821,6.076918e-05,9.635838,...,0.000000,0.000000,0.0,0.000000,388.997711,0.000191,0.952069,13.018297,2.305671e-04,83
54224501,361580,DE,B0BB7YSRBX,0.0,32.424000,43.990002,58.0,9.163816,6.362959e-05,9.159988,...,0.000000,0.000000,0.0,0.000000,388.911377,0.000175,0.952275,12.258468,1.078469e-04,93
54224502,361580,DE,B0BB7ZMGY8,0.0,32.424000,41.990002,452.0,11.256460,5.158017e-04,10.119755,...,0.000000,0.000000,0.0,0.000000,387.696472,0.000052,0.950042,13.149303,2.628407e-04,77


In [5]:
merged_candidates

,sess_id,sess_locale,product,target,sess_avg_price,product_price,product_freq,sasrec_scores_2,normalized_sasrec_scores_2,sasrec_scores_3,...,normalized_co_graph_counts_0,co_graph_counts_1,normalized_co_graph_counts_1,co_graph_counts_2,normalized_co_graph_counts_2,bert_scores,normalized_bert_scores,cos_bert_scores,sasrec_text_scores,normalized_sasrec_text_scores
0,0,UK,B000V599Y2,0.0,7.388571,5.200000,37.0,13.152878,7.433639e-04,10.677187,...,0.004992,0.000000,0.000000,0.0,0.000000,476.148773,8.255888e-06,0.965266,14.527957,2.660486e-04
1,0,UK,B007VZUA7U,0.0,7.388571,7.000000,36.0,9.393598,1.732076e-05,8.838863,...,0.004992,0.000000,0.000000,0.0,0.000000,475.083313,2.844725e-06,0.963742,13.181476,6.921363e-05
2,0,UK,B009EUAEQC,0.0,7.388571,7.490000,4.0,11.754339,1.835794e-04,10.670128,...,0.009983,1.033333,0.004797,0.0,0.000000,476.230896,8.962503e-06,0.964250,13.900270,1.420235e-04
3,0,UK,B00AH02IWG,0.0,7.388571,8.500000,3.0,12.194766,2.851667e-04,11.166204,...,0.006656,1.250000,0.005803,1.0,0.007937,472.108978,1.453126e-07,0.954584,12.074850,2.288707e-05
4,0,UK,B00I0UKKD4,0.0,7.388571,17.049999,118.0,11.835367,1.990737e-04,11.681271,...,0.004992,1.833333,0.008512,1.0,0.007937,482.222168,3.584311e-03,0.978083,17.833698,7.254814e-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77570148,361580,DE,B0BB7XV97M,0.0,32.424000,47.990002,56.0,9.117821,6.076918e-05,9.635838,...,0.000000,0.000000,0.000000,0.0,0.000000,388.997711,1.905752e-04,0.952069,13.018297,2.305671e-04
77570149,361580,DE,B0BB7YSRBX,0.0,32.424000,43.990002,58.0,9.163816,6.362959e-05,9.159988,...,0.000000,0.000000,0.000000,0.0,0.000000,388.911377,1.748122e-04,0.952275,12.258468,1.078469e-04
77570150,361580,DE,B0BB7ZMGY8,0.0,32.424000,41.990002,452.0,11.256460,5.158017e-04,10.119755,...,0.000000,0.000000,0.000000,0.0,0.000000,387.696472,5.187348e-05,0.950042,13.149303,2.628407e-04
77570151,361580,DE,B0BD4CP7N3,0.0,32.424000,24.990000,1.0,-3.778687,1.523355e-10,-1.612869,...,0.000000,0.000000,0.000000,0.0,0.000000,384.966888,3.384560e-06,0.941283,2.749540,8.000787e-09
